In [ ]:
import pandas as pd

In [ ]:
csv_path = '../data/articles_sources_removed.csv'

articles = pd.read_csv(csv_path)
articles.head()

In [ ]:
a = articles.iloc[0]['url']
a

In [ ]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'
}

response = requests.get(a, headers=headers)
response.status_code

In [ ]:
import bs4

soup = bs4.BeautifulSoup(response.content)
entry_text = soup.select('.entry__text')[0]
content = entry_text.select('.content-list-component')

full_text = []
for item in content:

    for el in item.contents:
        if isinstance(el, bs4.NavigableString):
            text = el.strip()
        else:
            text = el.text.strip()

        if text:
            full_text.append(text)

print('\n'.join(full_text))

In [ ]:
def scrape_contents(art: str):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'
    }

    response = requests.get(art, headers=headers)

    if response.status_code != 200:
        return None

    soup = bs4.BeautifulSoup(response.content)
    entry_text = soup.select('.entry__text')[0]
    content = entry_text.select('.content-list-component')

    full_text = []
    for item in content:
        for el in item.contents:
            if isinstance(el, bs4.NavigableString):
                text = el.strip()
            else:
                text = el.text.strip()

            if text:
                full_text.append(text)

    return '\n'.join(full_text)

In [ ]:
hp = articles.loc[articles['source'] == 'Huffington Post']

cntnt = []
for hp_a in hp.itertuples():
    cntnt.append(scrape_contents(hp_a[1]))

for cnt in cntnt:
    print(cnt, '\n\n\n')